In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/25 23:02:25 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.27.116 instead (on interface eth0)
22/08/25 23:02:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 23:02:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *


In [3]:
yellow_spark = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_indexed')

In [4]:
# create dataframes to aggregate data based in if it is a weekend or not and also aggregate based on win or lost

yellow_weekday = yellow_spark.filter( F.col("is_weekend") == True )
yellow_weekend = yellow_spark.filter( F.col("is_weekend") == False)

yellow_win = yellow_spark.filter( F.col("win") == "Yes" )
yellow_lose = yellow_spark.filter( F.col("win") == "No" )



In [5]:
# counts for wins 

yellow_win.count()

1276445

In [6]:
# counts for lost 
# higher number is expected as the knicks lost a more substantial amount of games this season compared
# to wins

yellow_lose.count()

4889389

In [7]:
# average tip amount per hour in a weekday
aggregate_weekday = (yellow_weekday
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )
aggregate_weekday.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekday')

In [8]:
# average tip amount per hour in a weekend

aggregate_weekend = (yellow_weekend
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_weekend.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_weekend')

In [9]:
# average tip amount per hour when the knicks win

aggregate_win = (yellow_win
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_win.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_win')

In [10]:
# average tip amount per hour when the knicks lose

aggregate_lose = (yellow_lose
     .groupBy(hour("tpep_pickup_datetime").alias("hour"))
     .agg(round(mean("tip_amount"),4).alias("mean_tip"))
     .orderBy("hour")
    )

aggregate_lose.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_lose')

In [11]:
# average tip amount based on when the game starts

aggregate_start = (yellow_spark
                       .groupBy(F.col("Start(ET)").alias("start_time"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("start_time"))
aggregate_start.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_start')

In [12]:
# aggregate by Pickup_location

aggregate_PU = (yellow_spark
                       .groupBy(F.col("PULocationID").alias("Pickup_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Pickup_location"))
aggregate_PU.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_PU')


In [13]:
aggregate_PU.show(24)

+---------------+--------+
|Pickup_location|mean_tip|
+---------------+--------+
|              1|   14.23|
|              2|   3.248|
|              3|  0.3266|
|              4|  2.2265|
|              5|     0.0|
|              6|    1.89|
|              7|  2.0128|
|              8|   2.235|
|              9|  0.1549|
|             10|  0.4327|
|             11|   0.263|
|             12|  2.8506|
|             13|  2.6931|
|             14|  0.6417|
|             15|  0.1198|
|             16|  0.3632|
|             17|  1.1741|
|             18|  0.3355|
|             19|  0.2311|
|             20|  0.2997|
|             21|  0.2797|
|             22|  0.5504|
|             23|  1.0133|
|             24|   2.101|
+---------------+--------+
only showing top 24 rows



In [14]:
# Aggregate by dropoff location

aggregate_DO = (yellow_spark
                       .groupBy(F.col("DOLocationID").alias("Dropoff_location"))
                       .agg(round(mean("tip_amount"),4).alias("mean_tip"))
                       .orderBy("Dropoff_location"))
aggregate_DO.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/aggregate_DO')

In [15]:
aggregate_DO.show(24)

+----------------+--------+
|Dropoff_location|mean_tip|
+----------------+--------+
|               1|  17.285|
|               2|    2.03|
|               3|  0.2409|
|               4|  2.1857|
|               6|     2.0|
|               7|  3.2945|
|               8|  3.0443|
|               9|  1.7292|
|              10|  2.6393|
|              11|  0.5524|
|              12|  2.8587|
|              13|  2.7433|
|              14|  2.9942|
|              15|   1.963|
|              16|  0.6199|
|              17|  2.9469|
|              18|  0.8039|
|              19|  0.1795|
|              20|  1.1351|
|              21|  0.8939|
|              22|   0.701|
|              23|  1.7314|
|              24|  2.2133|
|              25|  3.0389|
+----------------+--------+
only showing top 24 rows



In [16]:
# take 10% sample from main data

SAMPLE_SIZE = 0.10

yellow_sample = yellow_spark.sample(SAMPLE_SIZE, seed=0)

yellow_sample.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')
